In [1]:
pip install surprise

     |████████████████████████████████| 11.8 MB 2.6 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1630153 sha256=a3967bc7d4ed0be6a6536663ac1fa6900d2171250b48c82793d318201a61156a
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [3]:
%matplotlib inline
import pandas as pd
import numpy as np
from surprise import Reader
from surprise.accuracy import rmse
from surprise.model_selection import cross_validate

from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

from surprise import SVDpp
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise.model_selection import GridSearchCV



In [4]:
#Unzip the file
!unzip book1M.zip

Archive:  book1M.zip
   creating: book1M/
  inflating: book1M/BX-Book-Ratings.csv  
  inflating: book1M/BX-Books.csv     
  inflating: book1M/BX-Users.csv     
  inflating: book1M/explicit_ratings_books.csv  


In [5]:
#reading data
user = pd.read_csv('book1M/BX-Users.csv', sep=';', error_bad_lines=False, encoding="latin-1")
user.columns = ['userID', 'Location', 'Age']
ratings = pd.read_csv('book1M/BX-Book-Ratings.csv', sep=';', error_bad_lines=False, encoding="latin-1")
ratings.columns = ['userID', 'ISBN', 'Ratings']
df = pd.merge(user, ratings, on='userID', how='inner')
df.drop(['Location', 'Age'], axis=1, inplace=True)

df.head(10)

df.describe()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


,userID,Ratings
count,1.149780e+06,1.149780e+06
mean,1.403864e+05,2.866950e+00
std,8.056228e+04,3.854184e+00
min,2.000000e+00,0.000000e+00
25%,7.034500e+04,0.000000e+00
50%,1.410100e+05,0.000000e+00
75%,2.110280e+05,7.000000e+00
max,2.788540e+05,1.000000e+01


In [6]:
#preprocessing data
min_book_ratings = 50
filter_books = df['ISBN'].value_counts() > min_book_ratings
filter_books = filter_books[filter_books].index.tolist()

min_user_ratings = 50
filter_users = df['userID'].value_counts() > min_user_ratings
filter_users = filter_users[filter_users].index.tolist()

df_new = df[(df['ISBN'].isin(filter_books)) & (df['userID'].isin(filter_users))]
#df_new = df_new.fillna(0)
df_new.head(10)

,userID,ISBN,Ratings
394,243,0060915544,10
395,243,0060977493,7
397,243,0156006529,0
400,243,0316096199,0
401,243,0316601950,9
402,243,0316602906,0
403,243,0316666343,0
404,243,0316693006,0
405,243,0316776963,9
406,243,0316899984,7


In [7]:
user.head()

,userID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [8]:
ratings.head()

,userID,ISBN,Ratings
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [9]:
# trainset

reader = Reader()
data = Dataset.load_from_df(df_new[['userID','ISBN','Ratings']], reader)
trainset, testset = train_test_split(data, train_size=0.8, test_size=0.2)

type(data)

surprise.dataset.DatasetAutoFolds

In [11]:
algo_svdpp = SVDpp(n_factors=160, n_epochs=10, lr_all=0.005, reg_all=0.1)
algo_svdpp.fit(trainset)
test_pred = algo_svdpp.test(testset)
print("Hasil SVDpp")
accuracy.rmse(test_pred, verbose=True)

Hasil SVDpp
RMSE: 3.3931


3.3931030126007977

In [ ]:
cross_validate(algo_svdpp, data, measures=['RMSE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    3.3814  3.4015  3.4205  3.4122  3.4093  3.4050  0.0133  
MAE (testset)     2.8270  2.8349  2.8660  2.8506  2.8519  2.8461  0.0137  
Fit time          508.22  497.94  490.73  495.43  490.56  496.57  6.47    
Test time         7.57    7.12    7.20    6.56    6.86    7.06    0.34    


{'fit_time': (508.2179856300354,
  497.93669152259827,
  490.7271740436554,
  495.4255518913269,
  490.5618336200714),
 'test_mae': array([2.82702712, 2.83488231, 2.86599902, 2.85061789, 2.85186077]),
 'test_rmse': array([3.38140394, 3.401511  , 3.42051947, 3.41220156, 3.40930839]),
 'test_time': (7.5650975704193115,
  7.120659351348877,
  7.2032248973846436,
  6.557339906692505,
  6.862680435180664)}